# Updating exam_structure.json

This notebook is for updating, and adding new exams to the exam_structure.json file. This is mostly for updating the _structure_ attribute of the .json, as it is a nested dictonary and preserving the JSON format can be difficult.

In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
def json_format_preserver(df, json_filename):
    '''
    A shortcut way to make sure that the JSON file has its format preserved.
    Takes:
    df: a dataframe that wants to be converted to JSON format.
    json_filename: the filename of the JSON file. If it is not in the PWD, please
                   include the absolute filepath.

    Returns:
    None
    '''

    formatter = json.loads(df.to_json(orient='records'))

    with open(json_filename, 'w') as f:
        json.dump(formatter, f, indent=4)

The above function just helps with the tricky formatting.

Below we load in the JSON file.

In [3]:
exam_structure = pd.read_json('exam_details.json')

In [4]:
exam_structure

,name,gradebook,formlink,structure
0,Python Midterm,1LxSk2Kpbio9J7x9mxI5V3T0Cjx9POrTwLyLDWUNLXm8,https://docs.google.com/forms/d/e/1FAIpQLSc-bf...,{'Python Exam': [{'Question 1': ['Binary Searc...
1,Example Exam,1EslwwOrJuor0NyolZTmkQHe6Dc7oN2Ta3TZatuq_Qnc,https://docs.google.com/forms/d/e/1FAIpQLSdeOg...,{}
2,R Midterm,1u0Oy0LTGmMQxvjnL1s_pc1jho-eSp-kOgsmHR1cae00,https://docs.google.com/forms/d/e/1FAIpQLSfUfw...,{'R Exam': [{'File I/O': [{'Problem 1': ['List...
3,Python Machine Learning Midterm,1mzjHqTFNz2qfnOb9FKb74IuUe4GLDdqExHZscsIs2cA,https://docs.google.com/forms/d/e/1FAIpQLSfhrt...,{'Exploratory Data Analysis': [{'Subquestion 1...


The four attributes of any exam are as follows:

- __name__ : the name of the exam
- __gradebook__ : the ID of the google sheets containing the gradebook
- __formlink__  : the link to the google forms
- __structure__ : a nested dictonary of tuples that contains information about the exam.

As a quick example, if you have a gradebook with the ID 1LxSk2Kpbio9J7x9mxI5V3T0Cjx9POrTwLyLDWUNLXm8, then the link would be https://docs.google.com/spreadsheets/d/1LxSk2Kpbio9J7x9mxI5V3T0Cjx9POrTwLyLDWUNLXm8/

In [5]:
example_exam = {'name'      : 'Example Exam',
                'gradebook' : 'GRADEBOOK_ID',
                'formlink'  : 'FORM_LINK',
                'structure' : 'STRUCTURE_DICT'}

Below we go more indepth into how the actual exam structure is written. They are nested dictonaries with nested lists inside of them. The first key is always _the name of the exam_, which should key into a list where the first entry is the "section" (which in this case is the whole exam) and the second entry is the number of points this section is out of. Once we reach a list where the first item is a string, we can safely say that the string is a description of the question.

To highlight this in an example, we include two below. In the first one, we can see that we have a 'Question 1' as a key in our dict, that has the value \['An Example', 5\], where 'An Example' is a description for 'Question 1'.

Similarly, 'Subsection 1' has a dictonary as the first item in it's list, so it is technically a section. The second item is a section total. Since there is a 5 point question ('Subquestion 2') and a 10 point question ('Subquestion 1'), so the section total is 15 points.

We can see a real example below the toy example, included is the structure for the __R EDA exam__.

In [9]:
example_exam_str = {'Example Exam' : [{'Question 1' : ['An Example', 5],
                                       'Subsection 1' :
                                          [{'Subquestion 1' : ['Secondary Example', 10],
                                            'Subquestion 2' : ['Another Description', 5]}, 15]}, 20]}

In [11]:
r_structure = {'R Exam' : [{"File I/O" : [{"Problem 1" : ["Listing your Files", 5],
               "Problem 2" : ["Appending the Filepath", 5],
               "Problem 3" : ["Loading in your Data", 10]}, 20],
               "Wrangling" : [{"Problem 4" : ["Filtering your Data", 10],
               "Problem 5.a" : ["Cleaning Dates Part 1", 15],
               "Problem 5.b" : ["Cleaning Dates Part 2", 15],
               "Problem 6" : ["Cleaning Amounts", 15],
               "Problem 7" : ["Combining your Data", 15]}, 70],
               "Exploration" :[{"Problem 8" : ["Calculating Total Payments", 5],
               "Problem 9" : ["Calculating Coverage Period", 10],
               "Problem 10" : ["Calculating Average Annual Expenditure", 15],
               "Problem 11" : ["Cleaning Office and Purpose Columns", 15],
               "Problem 12" : ["Finding Purpose for the Spending", 25]}, 70]}, 160]}

Below is a function built on the same logic as how the reporter.py file interacts with the exam structure. You can test out the exam structures to make sure if they will be read properly by using the command.

In [7]:
def exam_str_parser(dictonary):
    for key in dictonary.keys():
        if type(dictonary[key][0]) == dict:
            print("Section: ", key)
            print("Out of", dictonary[key][1], "points")
            for item in dictonary[key][0].items():
                print(item[0], " : ", item[1][0])
        else:
            print("it doesn't happen")

In [8]:
exam_str_parser(example_exam_str)

Section:  Example Exam
Out of 15 points
Question 1  :  An Example
Subsection 1  :  {'Subquestion 1': ['Secondary Example', 10]}


In [12]:
example_exam['structure'] = example_exam_str

In [18]:
temp = exam_structure.loc[3, 'structure']

In [29]:
temp = {'Python Machine Learning Midterm': [temp, 100]}

In [27]:
exam_structure.at[3,'structure'] = temp

In [28]:
exam_structure

,name,gradebook,formlink,structure
0,Python Midterm,1LxSk2Kpbio9J7x9mxI5V3T0Cjx9POrTwLyLDWUNLXm8,https://docs.google.com/forms/d/e/1FAIpQLSc-bf...,{'Python Exam': [{'Question 1': ['Binary Searc...
1,Example Exam,1EslwwOrJuor0NyolZTmkQHe6Dc7oN2Ta3TZatuq_Qnc,https://docs.google.com/forms/d/e/1FAIpQLSdeOg...,{}
2,R Midterm,1u0Oy0LTGmMQxvjnL1s_pc1jho-eSp-kOgsmHR1cae00,https://docs.google.com/forms/d/e/1FAIpQLSfUfw...,{'R Exam': [{'File I/O': [{'Problem 1': ['List...
3,Python Machine Learning Midterm,1mzjHqTFNz2qfnOb9FKb74IuUe4GLDdqExHZscsIs2cA,https://docs.google.com/forms/d/e/1FAIpQLSfhrt...,{'Python Machine Learning Midterm': [{'Explora...


# Writing back to the file

We have it set up a basic pipeline in order to save the file back to itself. By default, we save it to a test.json file, you have to turn the "FINAL_SAVE" flag to TRUE in order to get the following section to work.

In [32]:
new_exam_structure = exam_structure.append(example_exam, ignore_index = True)

In [36]:
FINAL_SAVE = False

In [35]:
if FINAL_SAVE:
    filename = 'exam_details.json'
else:
    filename = 'test.json'

json_format_preserver(new_exam_structure, filename)